## Xarray with zarr and kerchunk

In [ ]:
import earthaccess
import fsspec
import xarray as xr
from common import earthaccess_args

In [ ]:
dataset = "gpm_imerg"
filename = earthaccess_args[dataset]["filename"]
bucket = earthaccess_args[dataset]["bucket"]
folder = earthaccess_args[dataset]["folder"]
variable = earthaccess_args[dataset]["variable"]
daac = earthaccess_args[dataset]["daac"]

In [ ]:
def load_data():
    if ".nc4" in filename:
        src = f"earthaccess_data/{filename[:-4]}.json"
    else:
        src = f"earthaccess_data/{filename[:-3]}.json"
    earthaccess.login()
    s3_fs = earthaccess.get_s3fs_session(daac=daac)
    storage_options = s3_fs.storage_options.copy()
    fsspec_caching = {
        "cache_type": "none",
    }
    fs = fsspec.filesystem("reference", fo=src, **fsspec_caching)
    m = fs.get_mapper("")
    da = xr.open_dataset(
        m,
        engine="kerchunk",
        chunks={},
        storage_options=storage_options,
    )[variable]
    return da.load()

In [ ]:
if __name__ == "__main__":
    da = load_data()